<a href="https://colab.research.google.com/github/Nico-Araujo/Enterprise-Challenge/blob/main/notebook/gs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Hard Hat Workers.v2-raw_75-25_traintestsplit.yolov8.zip to Hard Hat Workers.v2-raw_75-25_traintestsplit.yolov8.zip


In [ ]:
import zipfile
import os

# Pega o nome do arquivo ZIP enviado (a partir do dicionário 'uploaded')
zip_filename = list(uploaded.keys())[0]
zip_path = os.path.join('/content', zip_filename)

# Define o caminho onde o conteúdo será extraído
extract_path = '/content/hardhat_dataset'

# Cria a pasta de extração caso ela não exista
os.makedirs(extract_path, exist_ok=True)

# Extrai o arquivo ZIP para o diretório escolhido
with zipfile.ZipFile(zip_path, 'r') as z:
  z.extractall(extract_path)

print(f"'{zip_filename}' extracted to '{extract_path}/'")
print("Contents of the extracted directory:")
!ls {extract_path}


'Hard Hat Workers.v2-raw_75-25_traintestsplit.yolov8.zip' extracted to '/content/hardhat_dataset/'
Contents of the extracted directory:
data.yaml  README.dataset.txt  README.roboflow.txt  test  train


In [ ]:
with open('/content/hardhat_dataset/data.yaml', 'r') as f:
    data_yaml_content = f.read()

print(data_yaml_content)


names:
- head
- helmet
- person
nc: 3
roboflow:
  license: Public Domain
  project: hard-hat-workers
  url: https://universe.roboflow.com/joseph-nelson/hard-hat-workers/dataset/2
  version: 2
  workspace: joseph-nelson
test: ./test/images
train: ./train/images
val: ./test/images



In [ ]:
import yaml
import os

# Caminho para o arquivo data.yaml
DATA_YAML = '/content/hardhat_dataset/data.yaml'

# Ler o conteúdo atual do data.yaml
with open(DATA_YAML, 'r') as f:
    data = yaml.safe_load(f)

# Atualizar os caminhos para serem relativos à raiz do dataset (onde o data.yaml está)
data['train'] = './train/images'
# Como a pasta 'valid' está ausente, usaremos 'test' como conjunto de validação
data['val'] = './test/images'
data['test'] = './test/images'
# Escrever o conteúdo atualizado de volta para o data.yaml
with open(DATA_YAML, 'w') as f:
    yaml.dump(data, f)

print(f"Updated '{DATA_YAML}' with correct relative paths (using test for validation):")
with open(DATA_YAML, 'r') as f:
    print(f.read())

# Verificar a existência dos diretórios esperados
expected_train_path = os.path.join(os.path.dirname(DATA_YAML), data['train'])
expected_val_path = os.path.join(os.path.dirname(DATA_YAML), data['val'])
expected_test_path = os.path.join(os.path.dirname(DATA_YAML), data['test'])

print(f"\nChecking if training data path exists: {expected_train_path} -> {os.path.exists(expected_train_path)}")
print(f"Checking if validation data path exists: {expected_val_path} -> {os.path.exists(expected_val_path)}")
print(f"Checking if test data path exists: {expected_test_path} -> {os.path.exists(expected_test_path)}")

Updated '/content/hardhat_dataset/data.yaml' with correct relative paths (using test for validation):
names:
- head
- helmet
- person
nc: 3
roboflow:
  license: Public Domain
  project: hard-hat-workers
  url: https://universe.roboflow.com/joseph-nelson/hard-hat-workers/dataset/2
  version: 2
  workspace: joseph-nelson
test: ./test/images
train: ./train/images
val: ./test/images


Checking if training data path exists: /content/hardhat_dataset/./train/images -> True
Checking if validation data path exists: /content/hardhat_dataset/./test/images -> True
Checking if test data path exists: /content/hardhat_dataset/./test/images -> True


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# caminho para o data.yaml extraído do ZIP
DATA_YAML = '/content/hardhat_dataset/data.yaml'

model = YOLO('yolov8n.pt')  # modelo pré-treinado (vai baixar automaticamente)

# Treinamento
results = model.train(data=DATA_YAML, epochs=20, imgsz=640, batch=8, name='hardhat_detector_colab')
print('Training finished')

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/hardhat_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hardhat_detector_colab3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=

## Avaliação (val)

Roda a validação definida pelo `data.yaml` e mostra métricas (mAP, precision, recall).


In [ ]:
metrics = model.val()
print(metrics)


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1101.9±311.3 MB/s, size: 31.6 KB)
val: Scanning /content/hardhat_dataset/test/labels.cache... 1766 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1766/1766 3.3Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 111/111 7.4it/s 14.9s
                   all       1766       6808      0.966      0.621      0.661      0.463
                  head        339       1803      0.939      0.925      0.967      0.678
                helmet       1604       4863       0.96      0.939      0.983      0.692
                person         45        142          1          0     0.0319     0.0192
Speed: 0.8ms preprocess, 2.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/runs/detect/val
ul

##  Inferência - imagem, vídeo ou webcam

A célula abaixo faz inferência em imagens de teste.


In [ ]:
# exemplo de inferência em uma pequena amostra das imagens de teste
test_images_dir = '/content/hardhat_dataset/test/images'
test_list = []
import os
for f in os.listdir(test_images_dir)[:5]:
    if f.lower().endswith(('.jpg','.png')):
        test_list.append(os.path.join(test_images_dir,f))
print('Using examples:', test_list)

preds = model.predict(source=test_list, conf=0.5, save=True)
print('Predictions saved to runs/predict')


Using examples: ['/content/hardhat_dataset/test/images/006169_jpg.rf.fef7505d0e0933203ee6290c064f3c86.jpg', '/content/hardhat_dataset/test/images/006178_jpg.rf.a602d5d48bc4d54115f44de14fa664c0.jpg', '/content/hardhat_dataset/test/images/006564_jpg.rf.e1ad4b60a4a3891d1e4c47fc2f70a796.jpg', '/content/hardhat_dataset/test/images/006617_jpg.rf.9b7ee9eed6f8354aa3f7ac833c363b09.jpg', '/content/hardhat_dataset/test/images/006334_jpg.rf.2772c4e0ff02a2678a340d510686acf3.jpg']

0: 640x640 2 helmets, 11.2ms
1: 640x640 2 helmets, 11.2ms
2: 640x640 1 helmet, 11.2ms
3: 640x640 1 helmet, 11.2ms
4: 640x640 2 helmets, 11.2ms
Speed: 4.8ms preprocess, 11.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/detect/predict
Predictions saved to runs/predict


In [ ]:
from google.colab import files

uploaded = files.upload()
video_path = list(uploaded.keys())[0]

print("Vídeo carregado:", video_path)


Saving Nov_18__1758_23s_202511181810_qzvdd.mp4 to Nov_18__1758_23s_202511181810_qzvdd.mp4
Vídeo carregado: Nov_18__1758_23s_202511181810_qzvdd.mp4


**Teste com video feito por IA**

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/hardhat_detector_colab3/weights/best.pt')

results = model.predict(
    source=video_path,
    conf=0.5,
    save=True
)

print("Vídeo processado! Resultado salvo em:")
print(results[0].save_dir)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/552) /content/Nov_18__1758_23s_202511181810_qzvdd.mp4: 384x640 5 helmets, 39.3ms
video 1/1 (frame 2/552) /content/Nov_18__1758_23s_202511181810_qzvdd.mp4: 384x640 5 helmets, 9.4ms
video 1/1 (frame 3/552) /content/Nov_18__1758_23s_202511181810_qzvdd.mp4: 384x640 5 helmets, 8.2ms
video 1/1 (frame 4/552) /content/Nov_18__1758_23s_202511181810_qzvdd.mp4: 384x640 5 helmets, 12.5ms
video 1/1 (frame 5/552) /content/Nov_18__1758_23s_20251118

**Extraindo dados**

In [ ]:
import json
import os
from ultralytics import YOLO
from datetime import datetime

# CARREGAR O MODELO
model = YOLO('/content/runs/detect/hardhat_detector_colab3/weights/best.pt')

# DIRETÓRIO DE IMAGENS PARA TESTE
test_images_dir = "/content/hardhat_dataset/test/images"
output_json_dir = "/content/json_outputs"
os.makedirs(output_json_dir, exist_ok=True)

# LISTAR 5 IMAGENS DE EXEMPLO
images = [
    os.path.join(test_images_dir, f)
    for f in os.listdir(test_images_dir)
    if f.lower().endswith(('.jpg', '.png'))
][:5]

print("Imagens usadas:", images)

# PROCESSAR CADA IMAGEM
for img_path in images:
    results = model(img_path)
    r = results[0]

    detections_list = []

    # PEGAR TODAS AS DETECÇÕES DO YOLO
    for box in r.boxes:
        cls = model.names[int(box.cls[0])]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = box.xyxy[0].tolist()

        detections_list.append({
            "classe": cls,
            "confianca": conf,
            "bbox": [x1, y1, x2, y2]
        })

    # DETERMINANDO SE HÁ ALERTA (SEM CAPACETE)
    classes_detectadas = [d["classe"] for d in detections_list]

    if "person" in classes_detectadas and "helmet" not in classes_detectadas:
        alerta = {
            "tipo": "SEM_CAPACETE",
            "severidade": "ALTA",
            "mensagem": "Pessoa detectada sem capacete em zona obrigatória."
        }
    else:
        alerta = None

    # JSON FINAL NO FORMATO DO BD
    json_data = {
        "camera_id": 1,
        "frame": img_path,
        "timestamp": datetime.now().isoformat(),
        "face_borrada": 1,
        "deteccoes": detections_list,
        "alerta": alerta
    }

    # SALVAR JSON
    filename = os.path.basename(img_path).split('.')[0] + ".json"
    filepath = os.path.join(output_json_dir, filename)

    with open(filepath, "w") as f:
        json.dump(json_data, f, indent=4)

    print("Gerado:", filepath)


Imagens usadas: ['/content/hardhat_dataset/test/images/006169_jpg.rf.fef7505d0e0933203ee6290c064f3c86.jpg', '/content/hardhat_dataset/test/images/006178_jpg.rf.a602d5d48bc4d54115f44de14fa664c0.jpg', '/content/hardhat_dataset/test/images/006564_jpg.rf.e1ad4b60a4a3891d1e4c47fc2f70a796.jpg', '/content/hardhat_dataset/test/images/006617_jpg.rf.9b7ee9eed6f8354aa3f7ac833c363b09.jpg', '/content/hardhat_dataset/test/images/006334_jpg.rf.2772c4e0ff02a2678a340d510686acf3.jpg']

image 1/1 /content/hardhat_dataset/test/images/006169_jpg.rf.fef7505d0e0933203ee6290c064f3c86.jpg: 448x640 2 helmets, 40.4ms
Speed: 1.6ms preprocess, 40.4ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
Gerado: /content/json_outputs/006169_jpg.json

image 1/1 /content/hardhat_dataset/test/images/006178_jpg.rf.a602d5d48bc4d54115f44de14fa664c0.jpg: 480x640 2 helmets, 38.7ms
Speed: 2.0ms preprocess, 38.7ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Gerado: /content/json_outputs/006178